<a href="https://colab.research.google.com/github/TM0428/wsd2020/blob/main/system_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import time

def dirsName():
    return "data/"

def fileName(name: str, index: int, extension: str="html")->str:
    return dirsName() + name + "/" + str(index) + "." + extension

def urlName(article_url: str, article_index:int)->str:
    return article_url + str(article_index)

def main():
    contents = [ # name, domain, article_urlは手動でせざるを得ないか、article_indexを自動で取ってこられるようにしたい
        {
            "name": "sejuku",
            "domain":"https://www.sejuku.net/",
            "article_url":"https://www.sejuku.net/blog/",
            "article_index": [33294, 3766, 105909
            ] # 記事番号は https://www.sejuku.net/blog/curriculums-python から適当にとってきた。実際は https://www.sejuku.net/blog/archive/page/* から適当に取ってくるといいかもしれない
        },
        {
            "name": "techacademy",
            "domain":"https://techacademy.jp/",
            "article_url":"https://techacademy.jp/magazine/",
            "article_index": [15508, 15636, 15571] # 記事番号は https://techacademy.jp/magazine/category/programming/python/page/44 から適当にとってきた
        }
    ]
    for content in contents:
        print(content["name"])
        for index in content["article_index"]:
            filename = fileName(content["name"], index) # ローカル保存用のファイル名
            print(filename)
            print(os.path.exists(filename))
            if not os.path.exists(filename): # 1回getしたらローカルに保存して、複数回getしない
                urlname = urlName(content['article_url'], index) # get用のurl名
                print(urlname)
                html = requests.get(urlname) # getしたhtml(bytes)
                html_code = html.content.decode('utf-8') #getしたhtml(string)
                os.makedirs(dirsName(), exist_ok=True) # ディレクトリを作成
                os.makedirs(dirsName() + content["name"] + "/", exist_ok=True) # ディレクトリを作成
                with open(filename, mode='w', encoding='utf-8') as f:
                    f.write(html_code) # ローカルに保存
                time.sleep(3) # 適度なインターバル



if __name__=="__main__":
    main()


sejuku
data/sejuku/33294.html
False
https://www.sejuku.net/blog/33294
data/sejuku/3766.html
False
https://www.sejuku.net/blog/3766
data/sejuku/105909.html
False
https://www.sejuku.net/blog/105909
techacademy
data/techacademy/15508.html
False
https://techacademy.jp/magazine/15508


In [ ]:
# html to text
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

# text_setを生成する
# text_setは各葉ノードのテキストと木構造を表す
def soup2textset(tag, text_set, tag_string):
    for content in tag.contents:
        if isinstance(content, NavigableString):
            #content.string = content.string.replace('\xa9', '(c)').replace('\xa0', '')
            if not content.string in ["", "\n"]:
                #print(content.string)
                text_set.add(tag_string + content.string)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            pass
        else:
            #print(content.name)
            soup2textset(content, text_set, tag_string + str.lower(content.name))
    return text_set

# text_setに一致する葉ノードを削除する
def removetags(tag, remove_text_set, tag_string):
    remove_lists = []
    for content in tag.contents:
        if isinstance(content, NavigableString):
            if content.string in ["", "\n"] or (tag_string+content.string) in remove_text_set:
                remove_lists.append(content)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            remove_lists.append(content)
        else:
            removetags(content, remove_text_set, tag_string + str.lower(content.name))
    for content in remove_lists:
        content.extract()

# Webページを取得して解析する
# def html2text(htmls: list(str)) -> list(str):
# 同じドメインのhtml(string)群を受け取って、タグを削除したstringのlistを返す
def html2text(htmls: list[string]) -> list[string]:
    # 文字列抽出
    soups = [BeautifulSoup(html, "html.parser") for html in htmls]
    #bodies = [soup.find("body").text for soup in soups]

    bodies = [soup.find("body") for soup in soups]

    # 重複の削除
    if 2 <= len(bodies):
        # 重複要素の列挙
        remove_set = frozenset(soup2textset(bodies[0], set(), "") & soup2textset(bodies[1], set(), ""))
        #print(remove_set)
        # 重複要素の削除
        for i in range(len(bodies)):
            removetags(bodies[i], remove_set, "")

    #soup => textの変換
    for i in range(len(bodies)):
        bodies[i] = bodies[i].text

    # 空行削除
    emptyline_regex = re.compile('\n\\s*\n')
    for i in range(len(bodies)):
        bodies[i] = emptyline_regex.sub('\n', bodies[i])

    return bodies

def test():
    #load_url = "https://codezine.jp/article/detail/12230"
    #html = requests.get(load_url).content
    html = open("test.html", 'r', encoding="utf-8").read()
    html2 = open("test2.html", 'r', encoding="utf-8").read()
    bodies = html2text([html, html2])

    # print(bodies[0][:100])
    # print(bodies[0][-100:])
    # print("====================")
    # print(bodies[1][:100])
    # print(bodies[1][-100:])
    # print("====================")
    print(bodies[0].replace('\xa9', '(c)').replace('\xa0', ''))

if __name__=="__main__":
    test()



In [ ]:
import os


if __name__ == "__main__":
  # データ一覧
  print(os.listdir("./data"))
  

['techacademy_15571.html', 'sejuku_105909.html', 'sejuku_3766.html', 'techacademy_15508.html', 'techacademy_15636.html', 'sejuku_33294.html']
